# Test Notebook

In [15]:
import numpy as np
import pandas as pd
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from app.utils import clean_html_boilerplate

### Loading

In [25]:
DATA_DIR_PATH = "../data/"
LANG = "_DE"

TRAIN_PATH_CSV = DATA_DIR_PATH + "train" + LANG + ".csv"
TEST_PATH_CSV = DATA_DIR_PATH + "test" + LANG + ".csv"

N = 0

In [26]:
%%time
train = pd.read_csv(TRAIN_PATH_CSV)

CPU times: user 4.26 s, sys: 351 ms, total: 4.62 s
Wall time: 4.61 s


In [27]:
train2 = train.head(10)

### create test html

In [28]:
old_html = train.iloc[N]["html"]
old_html = old_html.replace("\n", "")
old_html = old_html.replace("\r", "")
old_html = old_html.replace("\t", "")
with open("../app/old/test.html", "w+") as f:
    f.write(old_html)

In [29]:
html_length = train.apply(lambda row: len(row["html"]), axis=1)
print(np.mean(html_length))

38347.23964318113


In [30]:
old_html[:100]

'<html lang="de">    <head>                                        <title>KSW Elektro- und Industriea'

## Only keep specific HTML Tags

In [31]:
from lxml.html.clean import Cleaner
from lxml import html, etree


html_file = etree.fromstring(old_html, parser=etree.HTMLParser(remove_comments=True))

tags = [element.tag for element in html_file.iter()]

In [ ]:
özel_tags = ["title", "h1", "h2", "h3", "b", "strong", "em", "i", "p", "a", "li"]

TODO:
- alles entfernen außer özel tags
    -  das vielleicht in cleaner?
    - mindestens die in `allow_tags`
        - https://lxml.de/api/lxml.html.clean.Cleaner-class.html
- paper 2020 hashemi weiterlesen
- weitere verfahren angucken
- clf pipeline aufsetzen

In [113]:
f = html.fromstring(old_html)

In [129]:
# get all tags

from lxml import etree
from lxml import html
import numpy as np

html_file = etree.fromstring(old_html, parser=etree.HTMLParser(remove_comments=True))

tags = [element.tag for element in html_file.iter()]
#list(np.unique(tags))

In [131]:
print(old_html)

<!doctype html><html class="no-js" lang=""><head><meta charset="utf-8"><meta http-equiv="x-ua-compatible" content="ie=edge"><meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no"><link rel="shortcut icon" type="image/x-icon" href="https://www.plersch.de/wp-content/themes/plersch/favicon.ico"><link rel="shortcut icon" type="image/png" href="https://www.plersch.de/wp-content/themes/plersch/favicon.png"><!-- This site is optimized with the Yoast SEO plugin v7.1 - https://yoast.com/wordpress/plugins/seo/ --><title>Start | Plersch Edelstahltechnik GmbH</title><link rel="canonical" href="https://www.plersch.de/" /><meta property="og:locale" content="de_DE" /><meta property="og:type" content="website" /><meta property="og:title" content="Start | Plersch Edelstahltechnik GmbH" /><meta property="og:description" content="Dies ist eine Beispiel-Seite. Sie unterscheidet sich von Beiträgen, da sie stets an der selben Stelle bleibt und (bei den meisten Themes) in der N

In [116]:
from lxml import etree
from lxml import html

f = html.fromstring(old_html)

# for r in f.xpath('//r'):
#     r.tail = 'DELETED' + r.tail if r.tail else 'DELETED'

etree.strip_elements(f, 'meta', with_tail=True)

print(etree.tostring(f,pretty_print=True)[:100])

b'<html class="no-js" lang="">\n  <head><link rel="shortcut icon" type="image/x-icon" href="https://www'


In [102]:
string = old_html

parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
parsed_xml = etree.fromstring(string.encode('utf-8'), parser=parser)
string = etree.tostring(parsed_xml)
string = string.decode("utf-8")

TypeError: Type 'NoneType' cannot be serialized.

## Clean HTML

### lxml

In [84]:
from lxml.html.clean import Cleaner
from lxml import html, etree


def clean_html_boilerplate(row):
    """ Cleans boilerplate HTML tags from HTML."""
    
    string = row.html
    url = row.url
    
    cleaner = Cleaner(page_structure=False, meta=True, style=True, kill_tags=["img"])
    clean = string
    
    try:
        clean = cleaner.clean_html(string)
    except:
        try:
            parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')
            parsed_xml = etree.fromstring(string.encode('utf-8'), parser=parser)
            string = etree.tostring(parsed_xml)
            string = string.decode("utf-8")
            clean = cleaner.clean_html(string)
        except:
            print(f"Website '{url}' couldn't be cleaned.")
    
    clean = clean.replace("\n", "")
    clean = clean.replace("\r", "")
    clean = clean.replace("\t", "")
    
    return clean

In [47]:
%%time
train["html"] = train.apply(lambda row: clean_html_boilerplate(row), axis=1)

CPU times: user 3min 22s, sys: 350 ms, total: 3min 22s
Wall time: 3min 22s


In [48]:
html_length = train.apply(lambda row: len(row["html"]), axis=1)
print(np.mean(html_length))

38466.64927477841


In [56]:
a = 119055
b = 38466

b/a-1

-0.6769056318508253

In [57]:
26 / 30

0.8666666666666667

In [7]:
cleaned_html = clean_html_boilerplate(old_html)
len(old_html), len(cleaned_html)

(27263, 13341)

### VERWORFEN: Tidylib (Python Wrapper)

In [30]:
from tidylib import Tidy
tidy = Tidy()
document, errors = tidy.tidy_document(cleaned_html, options={'alt-text': 'baz'})
document = document.replace("\n", "")
document = document.replace("\r", "")
document = document.replace("\t", "")

## real tests

In [15]:
cleaned_html = clean_html_boilerplate(old_html)

In [16]:
len(old_html), len(cleaned_html)

(27263, 13341)

In [106]:
with open("../app/old/test.html", "r") as f:
    testhtml = f.read()
    
    
html_doc = html.fromstring(soup_str)
html_doc_s = etree.tostring(html_doc, method="xml").decode("utf-8") 

In [113]:
print(html_doc_s[:1000])

<html class="no-js" lang="">
 <head>
  <title>
   Start | Plersch Edelstahltechnik GmbH
  </title>
 </head>
 <body class="home page-template-default page page-id-2">
  <aside class="pr-metaMenu">
   <nav>
    <ul class="pr-metaMenu__wrap" id="menu-meta-menue-kleingedrucktes-header">
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/impressum/">
       Impressum
      </a>
     </li>
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/kontakt/">
       Kontakt
      </a>
     </li>
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/datenschutz/">
       Datenschutz
      </a>
     </li>
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/agb/">
       AGB
      </a>
     </li>
    </ul>
   </nav>
  </aside>
  <header class="pr-siteHead">
   <div class="pr-siteHead__wrap">
   

In [99]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(cleaned_html)
soup_str = soup.prettify()
with open("../app/old/etest.html", "w+") as f:
    f.write(soup_str)

In [100]:
from lxml.html import html_to_xhtml
from lxml import etree

html_doc = html.fromstring(soup_str)
html_doc_s = etree.tostring(html_doc).decode("utf-8") 
print(len(html_doc_s))
print(html_doc_s[:1000])

with open("../app/old/etest.html", "w+") as f:
    f.write(html_doc_s)

16658
<html class="no-js" lang="">
 <head>
  <title>
   Start | Plersch Edelstahltechnik GmbH
  </title>
 </head>
 <body class="home page-template-default page page-id-2">
  <aside class="pr-metaMenu">
   <nav>
    <ul class="pr-metaMenu__wrap" id="menu-meta-menue-kleingedrucktes-header">
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/impressum/">
       Impressum
      </a>
     </li>
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/kontakt/">
       Kontakt
      </a>
     </li>
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/datenschutz/">
       Datenschutz
      </a>
     </li>
     <li class="pr-metaMenu__item-wrap">
      <a class="pr-metaMenu__item" href="https://www.plersch.de/agb/">
       AGB
      </a>
     </li>
    </ul>
   </nav>
  </aside>
  <header class="pr-siteHead">
   <div class="pr-siteHead__wrap

In [101]:
html_to_xhtml(html_doc)

html_doc_s = etree.tostring(html_doc).decode("utf-8") 
print(len(html_doc_s))
print(html_doc_s[:1000])

with open("../app/old/etest.xhtml", "w+") as f:
    f.write(html_doc_s)

18620
<html:html xmlns:html="http://www.w3.org/1999/xhtml" class="no-js" lang="">
 <html:head>
  <html:title>
   Start | Plersch Edelstahltechnik GmbH
  </html:title>
 </html:head>
 <html:body class="home page-template-default page page-id-2">
  <html:aside class="pr-metaMenu">
   <html:nav>
    <html:ul class="pr-metaMenu__wrap" id="menu-meta-menue-kleingedrucktes-header">
     <html:li class="pr-metaMenu__item-wrap">
      <html:a class="pr-metaMenu__item" href="https://www.plersch.de/impressum/">
       Impressum
      </html:a>
     </html:li>
     <html:li class="pr-metaMenu__item-wrap">
      <html:a class="pr-metaMenu__item" href="https://www.plersch.de/kontakt/">
       Kontakt
      </html:a>
     </html:li>
     <html:li class="pr-metaMenu__item-wrap">
      <html:a class="pr-metaMenu__item" href="https://www.plersch.de/datenschutz/">
       Datenschutz
      </html:a>
     </html:li>
     <html:li class="pr-metaMenu__item-wrap">
      <html:a class="pr-metaMenu__item" href="

In [9]:
train2["html"] = train2.apply(lambda row: clean_html_boilerplate(row["html"]), axis=1)

/Users/jan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
train.head(3)

,url,industry,industry_label,group,group_representative,html,text,source,country,group_representative_label
0,http://www.weldmont.de,56,Mining & Metals,man,55,<html> <head> ...,Weldmont\n\nHU\nDE\nEN\nSK\nHU\nDE\nEN\nSK\n\n...,xing,DE,Apparel & Fashion
1,http://www.rvs.at,41,Banking,fin,43,"<html lang=""de""><head> <title>Raiff...",Raiffeisenverband Salzburg Privatkunden\n\nSki...,xing,DE,Higher Education
2,http://www.unicon.at,11,Management Consulting,"corp, consulting",11,"<html lang=""de""><head><title>Unternehmensberat...","Unternehmensberatung, Coaching & Seminare | UN...",linkedin,DE,Motion Pictures and Film


In [44]:
gr = sorted(list(train.group_representative_label.unique()))
i = sorted(list(train.industry_label.unique()))

for element in gr:
    if element in i:
        print(element)

Hospitality
Logistics and Supply Chain
Management Consulting
Renewables & Environment


In [102]:
gr

['Apparel & Fashion',
 'Business Supplies and Equipment',
 'Civic & Social Organization',
 'Cosmetics',
 'Education Management',
 'Higher Education',
 'Hospitality',
 'Investment Banking',
 'Law Practice',
 'Logistics and Supply Chain',
 'Management Consulting',
 'Maritime',
 'Media Production',
 'Motion Pictures and Film',
 'Museums and Institutions',
 'Package/Freight Delivery',
 'Photography',
 'Renewables & Environment',
 'Research',
 'Wireless',
 'Writing and Editing']

In [11]:
train2.head(3)

,url,industry,industry_label,group,group_representative,html,text,source,country,group_representative_label
0,http://www.weldmont.de,56,Mining & Metals,man,55,<html> <head> ...,Weldmont\n\nHU\nDE\nEN\nSK\nHU\nDE\nEN\nSK\n\n...,xing,DE,Apparel & Fashion
1,http://www.rvs.at,41,Banking,fin,43,"<html lang=""de""><head> <title>Raiff...",Raiffeisenverband Salzburg Privatkunden\n\nSki...,xing,DE,Higher Education
2,http://www.unicon.at,11,Management Consulting,"corp, consulting",11,"<html lang=""de""><head><title>Unternehmensberat...","Unternehmensberatung, Coaching & Seminare | UN...",linkedin,DE,Motion Pictures and Film


In [14]:
with open("../app/old/test.html", "r") as f:
    old_html = f.read()

In [15]:
old_html[:100]

'<!DOCTYPE html><html>    <head>        <!-- Global site tag (gtag.js) - Google Analytics -->        '

In [17]:
cleaned_html = clean_html_boilerplate(old_html)

In [18]:
cleaned_html[:20]

'<html>    <head>    '

In [19]:
cleaned_html[-20:]

'      </body></html>'

In [20]:
from lxml import etree
from lxml import html
import numpy as np

html_file = html.fromstring(cleaned_html)

tags = [element.tag for element in html_file.iter()]
list(np.unique(tags))

['a',
 'body',
 'br',
 'div',
 'h1',
 'h2',
 'head',
 'html',
 'i',
 'li',
 'p',
 'span',
 'svg',
 'title',
 'ul']

In [21]:
# If you want to visit all of the descendants
for element in html_file.iter():
    print(element.tag)
    
    
# Or, if you want to have a list of all the descendents
all_elements = list(html_file.iter())
#print([element.tag for element in all_elements])

html
head
title
body
div
div
div
div
div
div
div
a
div
div
div
i
i
div
div
div
svg
div
a
div
a
div
a
div
a
div
div
a
svg
div
a
svg
div
a
svg
div
a
svg
div
div
div
div
div
a
svg
div
a
svg
div
a
svg
div
a
svg
div
div
svg
div
a
div
a
div
a
div
a
div
ul
li
a
li
a
div
div
div
a
div
div
div
div
h1
h2
div
svg
div
p
div
a
div
div
div
div
h1
h2
div
svg
div
p
div
div
div
div
div
div
span
br
br
br
div
span
br
br
br
div
div
div
ul
li
a
li
a
li
a
li
a
li
a
li
a
div
div
div
ul
li
a
li
a
li
a
li
a
li
a
div
div
div
div
ul
li
a
li
a
div
div
div
a
svg
div
a
svg
div
a
svg
div
a
svg
div
a
div
a
